In [67]:
import pandas as pd 
import sklearn as sk
import talib 
from talib import abstract
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt

In [2]:
label = pd.read_csv("10_label.csv")
margin = pd.read_csv("10_feature_past_6_months_margin_short.csv")
ohlcv = pd.read_csv("10_feature_past_6_months_ohlcv.csv")
trans = pd.read_csv("transaction_details.csv")
industry = pd.read_csv("stock_industry.csv")

In [3]:
label.describe()

,Unnamed: 0,stock_id,mfs_score,buy_price,sell_price,lowest_price,highest_price,profit(%),MAE,BMFE,GMFE
count,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000
mean,599.500000,4143.084167,82.963333,57.207758,58.399325,54.578208,61.510808,2.354568,4.463408,2.074393,7.748942
std,346.554469,2450.031282,3.096231,62.139442,63.418207,59.248870,66.885503,10.259694,5.711497,3.351196,8.971235
min,0.000000,1101.000000,73.000000,8.200000,8.710000,8.120000,9.040000,-29.272031,0.000000,0.000000,0.000000
25%,299.750000,2357.000000,81.000000,27.100000,27.200000,25.800000,28.400000,-3.312843,0.348743,0.000000,1.963034
50%,599.500000,3037.000000,83.000000,42.250000,42.375000,40.225000,44.900000,1.638226,2.393862,0.691976,4.850959
75%,899.250000,6156.750000,85.000000,66.400000,68.200000,63.625000,70.425000,6.964604,6.440489,2.827811,10.789187
max,1199.000000,9946.000000,92.000000,640.000000,640.000000,608.000000,659.000000,98.976982,39.034205,35.000000,98.976982


In [4]:
label.head(5)

,Unnamed: 0,stock_id,mfs_score,buy_price,sell_price,lowest_price,highest_price,buy_date,sell_date,profit(%),MAE,BMFE,GMFE
0,0,6108.0,82.0,24.15,24.30,24.00,24.6,2015-01-05,2015-01-09,0.621118,0.621118,0.000000,1.863354
1,1,8249.0,82.0,25.30,24.40,24.40,25.4,2015-01-05,2015-01-09,-3.557312,3.557312,0.395257,0.395257
2,2,2542.0,82.0,63.60,63.60,62.90,63.6,2015-01-05,2015-01-09,0.000000,1.100629,0.000000,0.000000
3,3,6220.0,80.0,18.95,18.65,18.55,19.0,2015-01-05,2015-01-09,-1.583113,2.110818,0.263852,0.263852
4,4,5514.0,80.0,21.30,21.45,20.95,21.6,2015-01-05,2015-01-09,0.704225,1.643192,0.000000,1.408451


In [125]:
margin.head(10)

,Unnamed: 0,stock_id,date,margin_balance,short_balance,margin_buy,margin_sell,margin_repayment,short_cover,short_sell,short_redemption
0,0,2387,2014-04-21,8106,52,452,284,0,3,0,0
1,1,2387,2014-04-22,8397,58,1307,1016,0,0,6,0
2,2,2387,2014-04-23,7870,59,725,1252,0,9,10,0
3,3,2387,2014-04-24,7471,59,435,834,0,0,0,0
4,4,2387,2014-04-25,6655,57,281,1097,0,2,0,0
5,5,2387,2014-04-28,6667,47,82,70,0,10,0,0
6,6,2387,2014-04-29,6698,46,144,113,0,1,0,0
7,7,2387,2014-04-30,6617,46,144,225,0,0,0,0
8,8,2387,2014-05-02,6443,49,95,269,0,0,3,0
9,9,2387,2014-05-05,6414,44,209,238,0,6,1,0


In [5]:
margin.describe()

,Unnamed: 0,stock_id,margin_balance,short_balance,margin_buy,margin_sell,margin_repayment,short_cover,short_sell,short_redemption
count,214782.000000,214782.000000,214782.000000,214782.000000,214782.000000,214782.000000,214782.000000,214782.000000,214782.000000,214782.000000
mean,894.575961,4142.924486,14271.323849,789.031828,698.876908,689.734894,10.437127,70.881191,76.748182,4.459498
std,516.935257,2449.305550,33107.714873,2739.053874,2275.211008,2188.560581,88.716070,323.572169,354.665863,70.454813
min,0.000000,1101.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,447.000000,2357.000000,3191.250000,18.000000,49.000000,56.000000,0.000000,0.000000,0.000000,0.000000
50%,894.000000,3037.000000,6519.000000,100.000000,172.000000,185.000000,0.000000,5.000000,4.000000,0.000000
75%,1342.000000,6153.000000,12347.000000,463.000000,536.000000,539.000000,2.000000,32.000000,31.000000,0.000000
max,1799.000000,9946.000000,421727.000000,64295.000000,83828.000000,95627.000000,9020.000000,16120.000000,17634.000000,7559.000000


In [6]:
trans.describe()

,Unnamed: 0,Net Worth,mdd,stock_id,mfs_score,buy_price,sell_price,lowest_price,highest_price,profit(%),MAE,BMFE,GMFE
count,2186.000000,2186.000000,2186.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000
mean,1092.500000,4.483783,-4.536380,4143.084167,82.963333,57.207758,58.399325,54.578208,61.510808,2.354568,4.463408,2.074393,7.748942
std,631.188165,3.096787,5.278437,2450.031282,3.096231,62.139442,63.418207,59.248870,66.885503,10.259694,5.711497,3.351196,8.971235
min,0.000000,0.917579,-33.833939,1101.000000,73.000000,8.200000,8.710000,8.120000,9.040000,-29.272031,0.000000,0.000000,0.000000
25%,546.250000,2.110139,-6.867975,2357.000000,81.000000,27.100000,27.200000,25.800000,28.400000,-3.312843,0.348743,0.000000,1.963034
50%,1092.500000,3.211393,-2.457276,3037.000000,83.000000,42.250000,42.375000,40.225000,44.900000,1.638226,2.393862,0.691976,4.850959
75%,1638.750000,7.178893,-0.663687,6156.750000,85.000000,66.400000,68.200000,63.625000,70.425000,6.964604,6.440489,2.827811,10.789187
max,2185.000000,13.262266,0.000000,9946.000000,92.000000,640.000000,640.000000,608.000000,659.000000,98.976982,39.034205,35.000000,98.976982


In [7]:
ohlcv.describe()

,Unnamed: 0,stock_id,open,close,high,low,volume,amount
count,215837.000000,215837.000000,215837.000000,215837.000000,215837.000000,215837.000000,2.158370e+05,2.158370e+05
mean,898.823432,4141.912485,52.647822,52.563068,53.318856,51.925848,7.301323e+06,4.394932e+08
std,519.228735,2448.236348,59.420255,59.330720,60.221386,58.580810,2.443221e+07,2.107578e+09
min,0.000000,1101.000000,4.850000,4.660000,4.960000,4.660000,1.000000e+03,1.542500e+04
25%,449.000000,2357.000000,24.550000,24.500000,24.850000,24.250000,6.130000e+05,1.993742e+07
50%,899.000000,3037.000000,38.050000,38.000000,38.500000,37.600000,1.888181e+06,7.190489e+07
75%,1348.000000,6153.000000,58.500000,58.500000,59.300000,57.700000,5.255000e+06,2.551090e+08
max,1799.000000,9946.000000,646.000000,633.000000,656.000000,630.000000,1.016328e+09,1.422155e+11


In [8]:
industry.describe()

,Unnamed: 0,stock_id,industry
count,1792.000000,1792.000000,1792.000000
mean,895.500000,4498.815848,12.067522
std,517.450159,2256.380317,7.175040
min,0.000000,1101.000000,0.000000
25%,447.750000,2535.750000,7.000000
50%,895.500000,4138.500000,12.000000
75%,1343.250000,6236.250000,18.000000
max,1791.000000,9962.000000,27.000000


In [9]:
pd.merge(ohlcv, industry, on="stock_id")

,Unnamed: 0_x,stock_id,date,open,close,high,low,volume,amount,Unnamed: 0_y,industry
0,0,2387,2014-04-18,14.75,14.60,14.80,14.45,1781758.0,26012241.0,314,15
1,1,2387,2014-04-21,14.50,14.55,14.65,14.45,996378.0,14484011.0,314,15
2,2,2387,2014-04-22,14.65,15.10,15.30,14.50,3433283.0,51333645.0,314,15
3,3,2387,2014-04-23,15.25,14.60,15.25,14.60,2591033.0,38409181.0,314,15
4,4,2387,2014-04-24,14.60,14.35,14.70,14.35,1262682.0,18348250.0,314,15
...,...,...,...,...,...,...,...,...,...,...,...
215832,1788,6290,2024-01-30,65.40,64.50,65.40,64.10,1178000.0,76121200.0,1548,7
215833,1789,6290,2024-01-31,64.60,64.50,65.10,64.20,974000.0,62994500.0,1548,7
215834,1790,6290,2024-02-01,64.90,64.70,65.10,64.50,1118000.0,72475900.0,1548,7
215835,1791,6290,2024-02-02,65.30,65.40,66.20,65.10,4409000.0,289973400.0,1548,7


In [ ]:
for i in range(len(industry["industry"].unique())):
    print (industry["industry"].unique()[i])


In [98]:
industry

,Unnamed: 0,stock_id,industry
0,0,1101,0
1,1,1102,0
2,2,1103,0
3,3,1104,0
4,4,1108,0
...,...,...,...
1787,1787,9949,27
1788,1788,9950,2
1789,1789,9951,8
1790,1790,9960,6


In [97]:
for i in range (len(industry["industry"].unique())):
    print(len(industry[industry["industry"] == industry["industry"].unique()[i]]))

138
29
25
83
35
51
23
206
95
16
127
41
47
28
30
106
174
87
93
113
35
40
19
44
42
18
22
25


In [11]:
label["mfs_score"].sort_values().unique()


array([73., 74., 75., 76., 77., 78., 79., 80., 81., 82., 83., 84., 85.,
       86., 87., 88., 89., 90., 91., 92.])

In [110]:
label_score = pd.merge(label, industry, on="stock_id")[["industry", "mfs_score","stock_id"]]


In [122]:
label

,Unnamed: 0,stock_id,mfs_score,buy_price,sell_price,lowest_price,highest_price,buy_date,sell_date,profit(%),MAE,BMFE,GMFE
0,0,6108.0,82.0,24.15,24.30,24.00,24.6,2015-01-05,2015-01-09,0.621118,0.621118,0.000000,1.863354
1,1,8249.0,82.0,25.30,24.40,24.40,25.4,2015-01-05,2015-01-09,-3.557312,3.557312,0.395257,0.395257
2,2,2542.0,82.0,63.60,63.60,62.90,63.6,2015-01-05,2015-01-09,0.000000,1.100629,0.000000,0.000000
3,3,6220.0,80.0,18.95,18.65,18.55,19.0,2015-01-05,2015-01-09,-1.583113,2.110818,0.263852,0.263852
4,4,5514.0,80.0,21.30,21.45,20.95,21.6,2015-01-05,2015-01-09,0.704225,1.643192,0.000000,1.408451
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,1195,3078.0,84.0,82.20,81.90,81.90,83.2,2024-02-15,2024-02-23,-0.364964,0.364964,1.216545,1.216545
1196,1196,2458.0,84.0,148.50,164.00,148.50,167.5,2024-02-15,2024-02-23,10.437710,0.000000,0.000000,12.794613
1197,1197,6284.0,84.0,79.30,78.70,78.70,81.8,2024-02-15,2024-02-23,-0.756620,0.756620,3.152585,3.152585
1198,1198,2355.0,82.0,46.80,50.50,46.80,52.0,2024-02-15,2024-02-23,7.905983,0.000000,11.111111,11.111111


In [121]:
label_score.sort_values(by="stock_id")

,industry,mfs_score,stock_id
501,0,86.0,1101.0
494,0,87.0,1101.0
480,0,91.0,1101.0
376,0,83.0,1101.0
468,0,83.0,1101.0
...,...,...,...
593,0,83.0,9945.0
497,0,86.0,9945.0
169,0,78.0,9945.0
764,0,84.0,9945.0


In [14]:
test[test["industry"] == 4][["mfs_score", "stock_id"]].groupby("stock_id").agg("mean").reset_index()


,stock_id,mfs_score
0,1319.0,80.466667
1,1522.0,79.000000
2,1524.0,79.000000
3,2204.0,84.200000
4,2206.0,76.333333
5,2227.0,84.000000
6,6605.0,81.000000


In [15]:
test[["mfs_score", "stock_id"]].groupby("stock_id").agg("mean").reset_index()


,stock_id,mfs_score
0,1101.0,83.750000
1,1102.0,84.333333
2,1104.0,83.000000
3,1108.0,85.500000
4,1109.0,88.000000
...,...,...
302,9927.0,83.666667
303,9939.0,80.666667
304,9943.0,83.500000
305,9945.0,83.000000


In [16]:
test[test["industry"] == 4][["mfs_score", "stock_id"]]["stock_id"].unique()


array([2204., 1319., 2227., 6605., 1522., 2206., 1524.])

In [17]:
for i in test[test["industry"] == 4][["mfs_score", "stock_id"]]["stock_id"].unique():
    print(i)
    print(test[test["stock_id"] == i]["mfs_score"].describe())

2204.0
count    15.000000
mean     84.200000
std       2.858571
min      80.000000
25%      82.000000
50%      83.000000
75%      86.000000
max      91.000000
Name: mfs_score, dtype: float64
1319.0
count    15.000000
mean     80.466667
std       2.503331
min      77.000000
25%      78.500000
50%      80.000000
75%      81.000000
max      86.000000
Name: mfs_score, dtype: float64
2227.0
count     1.0
mean     84.0
std       NaN
min      84.0
25%      84.0
50%      84.0
75%      84.0
max      84.0
Name: mfs_score, dtype: float64
6605.0
count     6.000000
mean     81.000000
std       4.049691
min      75.000000
25%      78.250000
50%      82.500000
75%      83.750000
max      85.000000
Name: mfs_score, dtype: float64
1522.0
count     3.000000
mean     79.000000
std       2.645751
min      76.000000
25%      78.000000
50%      80.000000
75%      80.500000
max      81.000000
Name: mfs_score, dtype: float64
2206.0
count     3.000000
mean     76.333333
std       1.154701
min      75.000000
25

In [19]:
pd.DataFrame({"技術名稱": list(talib.get_function_groups().keys()),
              "技術指標總數": list(map(lambda x: len(x), talib.get_function_groups().values()))})

,技術名稱,技術指標總數
0,Cycle Indicators,5
1,Math Operators,11
2,Math Transform,15
3,Momentum Indicators,30
4,Overlap Studies,17
5,Pattern Recognition,61
6,Price Transform,4
7,Statistic Functions,9
8,Volatility Indicators,3
9,Volume Indicators,3


In [20]:
talib.SMA(ohlcv[ohlcv["stock_id"] == 2330 ]["close"])

131611           NaN
131612           NaN
131613           NaN
131614           NaN
131615           NaN
             ...    
191363    482.183333
191364    483.600000
191365    485.583333
191366    487.883333
191367    490.500000
Length: 720, dtype: float64

In [21]:
df = ohlcv[ohlcv["stock_id"] == 2330].drop(columns=["stock_id","date"])
df = df.astype('float')
for i in talib.get_functions():
    try:
        result = eval('abstract.'+ i + '(ohlcv[ohlcv["stock_id"] == 2330])')
        result.name = i.lower() if type(result) == pd.core.series.Series else None
        df = pd.merge(df,pd.DataFrame(result),left_on= df.index,right_on=result.index)
        df = df.set_index("key_0")
    except:
        print(i)

MAVP


First calculate the avg mfs_score for each stock in label data 

In [22]:
score_avg = test [["mfs_score", "stock_id"]].groupby("stock_id").agg("mean").reset_index()

In [55]:
df_group = ohlcv.groupby("stock_id")
dfs = {}
for stock_id, grouped in df_group:
    dfs[stock_id] = grouped
    dfs[stock_id] = pd.merge(dfs[stock_id],score_avg, on="stock_id")
    dfs[stock_id] = dfs[stock_id].drop(columns=["stock_id","date"])
    dfs[stock_id] = dfs[stock_id].astype('float')



In [123]:
dfs[2330]

,Unnamed: 0,amount,mfs_score,ht_dcperiod,ht_dcphase,inphase,quadrature,sine,leadsine,ht_trendmode,...,linearreg_slope,stddev,tsf,var,atr,natr,trange,ad,adosc,obv
key_0,,,,,,,,,,,,,,,,,,,,,
0,360.0,1.608811e+10,78.0,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.368871e+07,NaN,5.475483e+07
1,361.0,1.267037e+10,78.0,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.5,6.170072e+05,NaN,9.767197e+07
2,362.0,1.229612e+10,78.0,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.5,-5.371698e+06,NaN,5.575104e+07
3,363.0,1.329984e+10,78.0,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.5,-1.042292e+07,NaN,1.029005e+07
4,364.0,1.038925e+10,78.0,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,2.063977,NaN,4.26,NaN,NaN,4.5,-3.250846e+06,NaN,-2.557031e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,715.0,1.704182e+10,78.0,28.738926,143.467252,44.974799,23.570744,0.595282,-0.147244,1,...,5.840659,7.797435,551.483516,60.80,12.258766,2.330564,16.0,-1.295304e+09,3.225313e+07,-1.275622e+09
716,716.0,1.276791e+10,78.0,29.863338,147.483100,42.876849,-41.329473,0.537548,-0.216152,1,...,4.657143,7.547185,547.214286,56.96,11.811711,2.258453,6.0,-1.311545e+09,1.960659e+07,-1.299984e+09
717,717.0,2.024793e+10,78.0,31.182875,150.999396,21.776674,-37.867469,0.484819,-0.275627,1,...,4.417582,8.009994,549.631868,64.16,12.182303,2.255982,17.0,-1.273912e+09,2.462478e+07,-1.262351e+09


In [124]:
ohlcv[ohlcv["stock_id"] == 2330 && ohlcv["date"] == "2020-01-02"]

,Unnamed: 0,stock_id,date,open,close,high,low,volume,amount
131611,360,2330,2019-10-15,293.5,293.5,296.0,292.0,54754830.0,1.608811e+10
131612,361,2330,2019-10-16,298.0,296.5,298.0,293.5,42917144.0,1.267037e+10
131613,362,2330,2019-10-17,295.0,293.5,295.5,292.0,41920938.0,1.229612e+10
131614,363,2330,2019-10-18,293.0,293.0,295.5,291.0,45460984.0,1.329984e+10
131615,364,2330,2019-10-21,291.0,290.0,291.0,288.5,35860365.0,1.038925e+10
...,...,...,...,...,...,...,...,...,...
191363,715,2330,2023-02-06,532.0,526.0,533.0,526.0,32208918.0,1.704182e+10
191364,716,2330,2023-02-07,524.0,523.0,528.0,522.0,24362116.0,1.276791e+10
191365,717,2330,2023-02-08,538.0,540.0,540.0,534.0,37633169.0,2.024793e+10
191366,718,2330,2023-02-09,540.0,540.0,540.0,536.0,22311345.0,1.202632e+10


In [57]:
for keys in dfs.keys():
    for i in talib.get_functions():
        try:
            result = eval('abstract.'+ i + '(dfs[keys])')
            result.name = i.lower() if type(result) == pd.core.series.Series else None
            dfs[keys] = pd.merge(dfs[keys],pd.DataFrame(result),left_on= dfs[keys].index,right_on=result.index)
            dfs[keys] = dfs[keys].set_index("key_0")
            # dfs[keys] = dfs[keys].drop(columns=["open","high","low","close","volume"])
        except:
            # print(i)
            continue


tunta: optimize the parameter 

In [58]:
for keys in dfs.keys():
    dfs[keys] = dfs[keys].drop(columns=["open","high","low","close","volume"])


In [89]:
df_2330 = dfs[2330]

In [90]:
i = 0
for col in df_2330.columns:
    if (df_2330[col].sum() == 0):
        df_2330 = df_2330.drop(columns=[col])
        i += 1

In [92]:
print(i)

29


In [93]:
dfs[2330]

,Unnamed: 0,amount,mfs_score,ht_dcperiod,ht_dcphase,inphase,quadrature,sine,leadsine,ht_trendmode,...,linearreg_slope,stddev,tsf,var,atr,natr,trange,ad,adosc,obv
key_0,,,,,,,,,,,,,,,,,,,,,
0,360.0,1.608811e+10,78.0,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.368871e+07,NaN,5.475483e+07
1,361.0,1.267037e+10,78.0,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.5,6.170072e+05,NaN,9.767197e+07
2,362.0,1.229612e+10,78.0,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.5,-5.371698e+06,NaN,5.575104e+07
3,363.0,1.329984e+10,78.0,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.5,-1.042292e+07,NaN,1.029005e+07
4,364.0,1.038925e+10,78.0,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,2.063977,NaN,4.26,NaN,NaN,4.5,-3.250846e+06,NaN,-2.557031e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,715.0,1.704182e+10,78.0,28.738926,143.467252,44.974799,23.570744,0.595282,-0.147244,1,...,5.840659,7.797435,551.483516,60.80,12.258766,2.330564,16.0,-1.295304e+09,3.225313e+07,-1.275622e+09
716,716.0,1.276791e+10,78.0,29.863338,147.483100,42.876849,-41.329473,0.537548,-0.216152,1,...,4.657143,7.547185,547.214286,56.96,11.811711,2.258453,6.0,-1.311545e+09,1.960659e+07,-1.299984e+09
717,717.0,2.024793e+10,78.0,31.182875,150.999396,21.776674,-37.867469,0.484819,-0.275627,1,...,4.417582,8.009994,549.631868,64.16,12.182303,2.255982,17.0,-1.273912e+09,2.462478e+07,-1.262351e+09


In [88]:
df_2330

,Unnamed: 0,amount,mfs_score,ht_dcperiod,ht_dcphase,inphase,quadrature,sine,leadsine,ht_trendmode,...,linearreg_slope,stddev,tsf,var,atr,natr,trange,ad,adosc,obv
key_0,,,,,,,,,,,,,,,,,,,,,
0,360.0,1.608811e+10,78.0,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.368871e+07,NaN,5.475483e+07
1,361.0,1.267037e+10,78.0,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.5,6.170072e+05,NaN,9.767197e+07
2,362.0,1.229612e+10,78.0,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.5,-5.371698e+06,NaN,5.575104e+07
3,363.0,1.329984e+10,78.0,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,4.5,-1.042292e+07,NaN,1.029005e+07
4,364.0,1.038925e+10,78.0,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,2.063977,NaN,4.26,NaN,NaN,4.5,-3.250846e+06,NaN,-2.557031e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,715.0,1.704182e+10,78.0,28.738926,143.467252,44.974799,23.570744,0.595282,-0.147244,1,...,5.840659,7.797435,551.483516,60.80,12.258766,2.330564,16.0,-1.295304e+09,3.225313e+07,-1.275622e+09
716,716.0,1.276791e+10,78.0,29.863338,147.483100,42.876849,-41.329473,0.537548,-0.216152,1,...,4.657143,7.547185,547.214286,56.96,11.811711,2.258453,6.0,-1.311545e+09,1.960659e+07,-1.299984e+09
717,717.0,2.024793e+10,78.0,31.182875,150.999396,21.776674,-37.867469,0.484819,-0.275627,1,...,4.417582,8.009994,549.631868,64.16,12.182303,2.255982,17.0,-1.273912e+09,2.462478e+07,-1.262351e+09


In [62]:
df_train = pd.DataFrame()
for keys in dfs.keys():
    df_train = pd.concat([df_train,dfs[keys]])

In [63]:
df_train

,Unnamed: 0,amount,mfs_score,ht_dcperiod,ht_dcphase,inphase,quadrature,sine,leadsine,ht_trendmode,...,linearreg_slope,stddev,tsf,var,atr,natr,trange,ad,adosc,obv
key_0,,,,,,,,,,,,,,,,,,,,,
0,0.0,244549754.0,83.75,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-6.198285e+05,NaN,6818114.0
1,1.0,260749525.0,83.75,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.00,6.532687e+06,NaN,13970630.0
2,2.0,218271344.0,83.75,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.60,1.046676e+07,NaN,19871732.0
3,3.0,153826880.0,83.75,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.40,1.462653e+07,NaN,24031511.0
4,4.0,115471555.0,83.75,NaN,NaN,NaN,NaN,NaN,NaN,0,...,NaN,0.467547,NaN,0.2186,NaN,NaN,0.60,1.399823e+07,NaN,20889999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,1794.0,76094420.0,80.00,21.964509,244.124324,-0.122765,0.370729,-0.899743,-0.944810,1,...,0.021978,0.474974,23.793407,0.2256,0.496394,2.026096,0.60,-1.837350e+07,-1.820750e+06,46950681.0
176,1795.0,82151350.0,80.00,21.737413,242.173906,0.010270,0.447877,-0.884368,-0.955413,1,...,0.046374,0.483322,24.026374,0.2336,0.518080,2.123278,0.80,-2.127802e+07,-2.773356e+06,43631232.0
177,1796.0,119012162.0,80.00,21.311339,139.284827,0.072062,0.447868,0.652299,-0.074715,1,...,0.103187,0.739189,24.598901,0.5464,0.595360,2.316575,1.60,-1.990368e+07,-2.473646e+06,48303981.0
